# MERFISH Analysis Code for "Intratumoral mregDC/CXCL13 T helper niches enable local differentiation of CD8 effector T cells following PD-1 blockade"

Assaf Magen, Pauline Hamon, Nathalie Fiaschi, Brian Y. Soong, Matthew D. Park, Raphal Mattiuz, Etienne Humblin, Leanna Troncoso, Darwin Dsouza, Travis Dawson, Joel Kim, Steven Hamel, Mark Buckup, Christie Chang, Alexandra Tabachnikova, Hara Schwartz, Nausicaa Malissen, Yonit Lavin, Alessandra SoaresSchanoski, Bruno Giotti, Samarth Hegde, Clotilde Hennequin, Jessica Le Berichel, Zhen Zhao, Stephen Ward, Isabel Fiel, Baijun Kou, Michael Dobosz, Lianjie Li, Christina Adler, Min Ni, Yi Wei, Wei Wang, Namita T. Gupta, Gurinder Atwal, Kunal Kundu, Kamil Cygan, RaquelP. Deering, Alex Tsankov, Adeeb Rahman, Colles Price, Nicolas Fernandez, Jiang He, Seunghee KimSchulze, Sacha Gnjatic, Ephraim Kenigsberg, Myron Schwartz, Thomas U. Marron, Gavin Thurston, Alice O. Kamphorst, Miriam Merad

# Imports

In [ ]:
import anndata as ad
import cnmf
import gc
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from scipy.sparse import csr_matrix
import seaborn as sns
import scanpy as sc
import squidpy as sq
import scvi

In [ ]:
base_path = ### Insert base path 

anndatas_path = base_path + "AnnDatas/"
data_path = base_path + "Internal_HCC_MERFISH_From_Vizgen/analyzed_data/"
models_path = base_path + "scVI_Models/"
plots_path = base_path + "Plots/"
observable_pickle_path = base_path + "Observable_Pickle/"

metadata_path = base_path + "metadata.csv"

# Preprocess

## Import all raw MERFISH data and create AnnDatas

In [ ]:
patients = os.listdir(data_path)
patients = [patient + "/" for patient in patients]

In [ ]:
ids = []

for patient in patients:
    if patient[0] == "H":
        id = patient.split("_")
        id = "_".join(id[:-1])
    else:
        id = patient.split("_")[1]
        id = id.replace("-", "_")
    
    ids.append(id)

In [ ]:
for patient in patients:
    print("Converting", patient)
    
    regions = os.listdir(data_path + patient)
    regions = [region + "/" for region in regions]
    
    if patient[0] == "H":
        data_id = patient.split("_")
        data_id = "_".join(data_id[:-1])
    else:
        data_id = patient.split("_")[1]
        data_id = data_id.replace("-", "_")
    
    for region in regions:
        try:
            full_path = data_path + patient + region

            hcc_pt_cell_by_gene = pd.read_csv(full_path + "Cellpose/cellpose_cell_by_gene.csv", index_col = 0)
            hcc_pt_cell_metadata = pd.read_csv(full_path + "Cellpose/cellpose_cell_metadata.csv", index_col = 0)

            ad_hcc_pt = ad.AnnData(
                X = csr_matrix(hcc_pt_cell_by_gene.values),
                obs = hcc_pt_cell_metadata,
                var = pd.DataFrame(index = hcc_pt_cell_by_gene.columns)
            )

            ad_hcc_pt.obsm["spatial"] = ad_hcc_pt.obs[["center_x", "center_y"]].values

            ad_hcc_pt.obs["data_id"] = data_id
            ad_hcc_pt.obs["region"] = region[:-1]
            ad_hcc_pt.obs["barcodeCount"] = ad_hcc_pt.X.sum(axis = 1)

            ad_hcc_pt.var["expression"] = ad_hcc_pt.X.sum(axis = 0).T

            ad_hcc_pt.write_h5ad(anndatas_path + "HCC_MERFISH_Internal_" + data_id + "_" + region[:-1] + "_ad_counts.h5ad")

            print("Saved", data_id, region[:-1])

        except Exception as e:
            print("Skipped", data_id, region[:-1])
            print(e)
            continue 

## Generate a metadata.csv file for AnnDatas

In [ ]:
metadata = []

patients = os.listdir(data_path)

ids = []

for patient in patients:
    if patient[0] == "H":
        id_split = patient.split("_")
        id = "_".join(id_split[:-1])
        date = "_".join(id_split[-1:])
        method = "OCT"
        
    else:
        id = patient.split("_")[1]
        id = id.replace("-", "_")
        date = patient.split("_")[0]
        method = "FFPE"
    
    ids.append(id)
    
    regions = os.listdir(data_path + patient)
    
    for region in regions:
        full_path = data_path + patient + "/" + region + "/"
        
        if "Cellpose" in os.listdir(full_path):
            cellpose = True
        else:
            cellpose = False
            
        if "122" in id:
            treatment = True
        else:
            treatment = False
            
        if "Sample" in id:
            pretreatment = True
        else:
            pretreatment = False

        metadata.append(
            {
                "data_id" : id,
                "region" : region,
                "date" : date,
                "full_path" : full_path,
                "cellpose" : cellpose,
                "method" : method,
                "treatment" : treatment,
                "pretreatment" : pretreatment
            }
    )

In [ ]:
df_metadata = pd.DataFrame.from_records(metadata)
df_metadata

In [ ]:
df_metadata.to_csv(base_path + "metadata.csv")

# scVI Integration

In [ ]:
df_metadata = pd.read_csv(metadata_path, index_col = 0)
df_metadata = df_metadata[df_metadata["cellpose"] == True]
df_metadata

In [ ]:
ad_pts = {}

for i, row in df_metadata.iterrows():
    label = row["data_id"] + "_" + row["region"]
    
    ad_pt = ad.read_h5ad(anndatas_path + "HCC_MERFISH_Internal_" + label + "_ad_counts.h5ad")
    
    ad_pt.obs["method"] = row["method"]
    ad_pt.obs["treatment"] = row["treatment"]
    
    ad_pts[label] = ad_pt

In [ ]:
ad_int = ad.concat(ad_pts, index_unique = "_", label = "dataset", uns_merge = "same")

In [ ]:
ad_int = ad_int[:, ~ad_int.var.index.str.contains("Blank")]

In [ ]:
expression = ad_int.X.sum(axis = 0)
barcodeCount = ad_int.X.sum(axis = 1)

ad_int.var["expression"] = np.array(expression).flatten()
ad_int.obs["barcodeCount"] = barcodeCount

In [ ]:
percentile = 95

In [ ]:
ad_int = ad_int[:, ad_int.var["expression"] <= np.percentile(ad_int.var["expression"], percentile)]

In [ ]:
min_gene_per_cell = 5

min_count_per_cell = 10
max_count_per_cell = 1250

min_vol_per_cell = 100
max_vol_per_cell = 3000

In [ ]:
ad_int = ad_int[(ad_int.obs["volume"] > min_vol_per_cell) & (ad_int.obs["volume"] < max_vol_per_cell), :]

In [ ]:
sc.pp.filter_cells(ad_int, min_counts = min_count_per_cell)

sc.pp.filter_cells(ad_int, max_counts = max_count_per_cell)

sc.pp.filter_cells(ad_int, min_genes = min_gene_per_cell)

In [ ]:
ad_int.layers["counts"] = ad_int.X.copy()

sc.pp.normalize_total(ad_int, target_sum = 1e4)
sc.pp.log1p(ad_int)
ad_int.raw = ad_int

In [ ]:
forbidden_genes = pd.read_csv(base_path + "scRNAseq_From_Assaf/forbidden_genes.txt")["Gene"].str.upper()

In [ ]:
ad_int.var["forbidden_gene"] = ad_int.var_names.isin(forbidden_genes)

In [ ]:
ad_int = ad_int[:, ad_int.var["forbidden_gene"] == False]

In [ ]:
sc.pp.highly_variable_genes(
    ad_int,
    flavor = "seurat_v3",
    n_top_genes = ad_int.shape[1],
    layer = "counts",
    batch_key = "dataset",
    subset = True
)

In [ ]:
scvi.model.SCVI.setup_anndata(
    ad_int, 
    layer = "counts", 
    batch_key = "dataset",
    categorical_covariate_keys = ["method", "treatment"]
)

In [ ]:
model = scvi.model.SCVI(ad_int)

In [ ]:
model.train(use_gpu = 1, max_epochs = 100)

# Scanpy clustering

In [ ]:
n_neighbors = 10

print("model normalized expression")

normalized_expression = model.get_normalized_expression()
ad_int.layers["scVI_normalized"] = normalized_expression

print("neighbors")

sc.pp.neighbors(ad_int, use_rep = "X_scVI", n_neighbors = n_neighbors)

n_pcs = 30
min_dist = 0.1
spread = 3.0

print("umap")

sc.tl.umap(ad_int, min_dist = min_dist, spread = spread)

ad_int.write_h5ad(anndatas_path + experiment_name + "_n_neigh_" + str(n_neighbors) + ".h5ad")

In [ ]:
resolutions = [2.0]

for resolution in resolutions:
    print(resolution)
    
    sc.tl.leiden(ad_int, resolution = resolution, key_added = "leiden_" + str(resolution))
    
ad_int.write_h5ad(anndatas_path + experiment_name + "_n_neigh_" + str(n_neighbors) + "_res_0.8.h5ad")


# Subclustering

## Gene-gene correlation

## Clustering based on subset of genes + cells

# Proximity Analysis

In [ ]:
### Import anndata
print("importing")

ad_int_immune = ad_int_immune[~ad_int_immune.obs["leiden_2.0"].isin(["36", "37"]), :]

sorted_datasets = ad_int_immune.obs["dataset"].value_counts().sort_values().index.values

for dataset in sorted_datasets:
    
    print(dataset)
    
    ### Subset data
    print("subsetting")

    ad_int_immune_dataset = ad_int_immune[ad_int_immune.obs["dataset"] == dataset, :].copy()

    ### Center sample
    print("centering")

    spatial_dataset = ad_int_immune_dataset.obsm["spatial"]

    spatial_dataset[:, 0] -= np.mean(spatial_dataset[:, 0])
    spatial_dataset[:, 1] -= np.mean(spatial_dataset[:, 1])

    ad_int_immune_dataset.obsm["spatial"] = spatial_dataset

    ### Plotting UMAP for colors
    sc.pl.umap(ad_int_immune_dataset, color = "leiden_2.0")

    ### Co-occurrence analysis
    print("co-occurrence")

    intervals = np.arange(25, 1000, 25)

    sq.gr.co_occurrence(ad_int_immune_dataset, n_splits = 0, cluster_key = "leiden_2.0", interval = intervals)

    co_ocurrence = ad_int_immune_dataset.uns["leiden_2.0_co_occurrence"]

    print("saving pickle")

    with open(f"co_courrence_cluster_all_{dataset}.pickle", "wb") as handle:
        pickle.dump(co_ocurrence, handle, protocol = pickle.HIGHEST_PROTOCOL)

    print("saving plot")
    mpl.rcParams["figure.dpi"] = 300

    sq.pl.co_occurrence(ad_int_immune_dataset, cluster_key = "leiden_2.0", clusters = ["15", "34"], figsize = (20, 10))

    plt.savefig(f"co_ocurrence_cluster_15_cluster_34_{dataset}.png")


In [ ]:
datasets = ad_int_immune.obs["dataset"].cat.categories

def save_cluster_data(dataset, co_ocurrence_data, cluster_num):
    df = pd.DataFrame(
        co_ocurrence_data["occ"][:, cluster_num, :].T, index = co_ocurrence_data["interval"][1:]
    )
    
    df.to_csv(f"co_ocurrence_cluster_{cluster_num}_{dataset}.csv")
    
    print(f"Saved {dataset} cluster {cluster_num}")
        

for dataset in datasets:
    with open(f"co_courrence_cluster_all_{dataset}.pickle", "rb") as handle:
        co_occurrence = pickle.load(handle)
            
    save_cluster_data(dataset, co_occurrence, 15)
    save_cluster_data(dataset, co_occurrence, 34)

# cNMF Analysis

## Generate commands to run cNMF on LSF cluster

In [ ]:
base_path = "/sc/arion/projects/Merad_Lab/soongb02/Merad_Lab/Project_HCC/MERFISH/"
cnmf_script_path = "/hpc/users/soongb02/ji_lab/20220726_cnmf.py"

# General arguments
output_dir = base_path + "cNMF"
name = "20220920_HCC_MERFISH_Merge_scVI_model_Internal_All_7_with_Treatment_Immune_cell_subintegration_subclustering_cluster_15_cNMF_2000_iter"
counts = cnmf_path + experiment_name + "_n_neigh_" + str(n_neighbors) + "_res_2.0_cluster_15_only_for_cNMF.h5ad"
k_range = " ".join([str(k) for k in np.arange(2, 20)])
n_iter = 100
seed = 123456
n_workers = 500

# Factorize arguments
path = "/sc/arion/work/soongb02/miniconda3/envs/cnmf_env/bin/"
output_file = output_dir + "/bsub_outputs/" + name + "_\%I.output"

account = "acc_Merad_Lab"
queue = "premium"
n_cores = 1
mem = 3000
wall_time = "24:00"

# Consensus arguments
components = 11
local_density_threshold = 0.1

prepare_cmd = (
    "cnmf prepare --output-dir {output_dir} --name {name} -c {counts} --tpm {tpm} -k {k_range} --n-iter {n_iter} --seed {seed} --total-workers {n_workers} {densify}"
    .format(
        output_dir = output_dir,
        name = name, 
        counts = counts, 
        tpm = counts,
        k_range = k_range, 
        n_iter = n_iter, 
        seed = seed,
        n_workers = n_workers,
        densify = "--densify"
    )
)

factorize_cmd = (
    "{path}cnmf factorize --output-dir {output_dir} --name {name} --worker-index $LSB_JOBINDEX --total-workers {n_workers}"
    .format(
        path = path,
        output_dir = output_dir,
        name = name,
        n_workers = n_workers
    )
)

factorize_bsub_cmd = (
    "bsub -P {account} -q {queue} -n {n_cores} -M {mem} -W {wall_time} -o {output_file} -J cNMF[1-{n_workers}]"
    .format(
        account = account,
        queue = queue,
        n_cores = n_cores,
        mem = mem,
        wall_time = wall_time,
        output_file = output_file,
        n_workers = n_workers
    )
)

factorize_full_cmd = factorize_bsub_cmd + " '" + factorize_cmd + "'"

combine_cmd = (
    "cnmf combine --output-dir {output_dir} --name {name}"
    .format(
        output_dir = output_dir,
        name = name
    )
)

k_selection_plot_cmd = (
    "cnmf k_selection_plot --output-dir {output_dir} --name {name}"
    .format(
        output_dir = output_dir,
        name = name
    )
)

consensus_cmd = (
    "python {cnmf_script_path} consensus --output-dir {output_dir} --name {name} --components {components} --local-density-threshold {local_density_threshold} --show-clustering"
    .format(
        cnmf_script_path = cnmf_script_path,
        output_dir = output_dir,
        name = name,
        components = components, 
        local_density_threshold = local_density_threshold
    )
)

def print_new(string, header = False):
    if header == True:
        repeat = 10
        equals_line = " " + "=" * repeat + " "
        
        string = equals_line + string + equals_line
    
    print(string + "\n")

command_names = [i + " command" for i in ["prepare", "factorize", "combine", "k_selection_plot", "consensus"]]
commands = [prepare_cmd, factorize_full_cmd, combine_cmd, k_selection_plot_cmd, consensus_cmd]
    
for command_name, command in zip(command_names, commands):
    print_new(command_name, header = True)
    print_new(command)

## Export cNMF results

In [ ]:
components = 33
local_density_threshold = 0.18
name = "20220920_HCC_MERFISH_Merge_scVI_model_Internal_All_7_with_Treatment_Immune_cell_subintegration_subclustering_cNMF_2000_iter"

cnmf_obj = cnmf.cNMF(output_dir = output_dir, name = name)
usage, spectra_scores, spectra_tpm, top_genes = cnmf_obj.load_results(K = components, density_threshold = local_density_threshold)

In [ ]:
usage.to_csv(f"{cnmf_path}{name}/k_{components}_dt_{local_density_threshold}_usage.csv")

In [ ]:
spectra_scores.to_csv(f"{cnmf_path}{name}/k_{components}_dt_{local_density_threshold}_spectra_scores.csv")

In [ ]:
top_genes.to_csv(f"{cnmf_path}{name}/k_{components}_dt_{local_density_threshold}_top_genes.csv")